<img src="http://openenergy-platform.org/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>

# Usage of OpenEnergyPlatform API-Dialect (oedialect)
Repository: https://github.com/openego/oedialect <br>
Documentation: http://oep-data-interface.readthedocs.io/en/latest/api/how_to.html

Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/oedialect/issues <br>

In [ ]:
__copyright__ = "Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "henhuy, Ludee, oakca"

In [ ]:
import pandas as pd
import getpass
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
import oedialect

## Connection to OEP

If we want to upload data to the OEP we first need to connect to it, using our OEP user name and token. 
Note: You ca view your token on your OEP profile page after logging in.

In [ ]:
# Whitespaces are not a problem for setting up the url!
user = input('Enter OEP-username:')
token = getpass.getpass('Token:')

In [ ]:
# Create Engine:
OEP_URL = 'openenergy-platform.org'
OED_STRING = f'postgresql+oedialect://{user}:{token}@{OEP_URL}'

engine = sa.create_engine(OED_STRING)
metadata = sa.MetaData(bind=engine)

## Setup a Table

We need to tell the data base what columns and datatypes we are about to upload. In our case we have four columns, two of which are text, one is integer and the last is float.

In [ ]:
tname = 'example_dialect_table'
sname = 'sandbox'

table = sa.Table(
    tname,
    metadata,
    sa.Column('name', sa.VARCHAR(50)),
    sa.Column('age', sa.INTEGER),
    schema=sname
)

## Create the new Table

Now we tell our engine to connect to the data base and create the defined table within the chosen schema.

In [ ]:
conn = engine.connect()
print('Connection established')
if not engine.dialect.has_table(conn, tname, sname):
    table.create()
    print('Created table')

## Insert data into Table
 
Uploading the information from our DataFrame is now done with a single command. Uploading data in this way will always delete the content of the table and refill it with new values every time. If you change 'replace' to 'append', the data entries will be added to the preexisting ones. (Connecting and uploading may take a minute.)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    insert_statement = table.insert().values(
        [
            dict(name='Peter', age=25),
            dict(name='Inge', age=42),
            dict(name='Horst', age=36)
        ]
    )
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()
    raise
finally:
    session.close()

## Select from Table

Now  we can query our table to see if the data arrived.

In [ ]:
# ORM classes -> ego.io
from sqlalchemy import BigInteger, Boolean, Column, Date, Float, Integer, Numeric, String, Table, Text, text
from geoalchemy2.types import Geometry
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata

class BkgVg2502Lan(Base):
    __tablename__ = 'bkg_vg250_2_lan'
    __table_args__ = {'schema': 'boundaries'}

    reference_date = Column(Date, primary_key=True, nullable=False)
    id = Column(Integer, primary_key=True, nullable=False, server_default=text("nextval('boundaries.bkg_vg250_2_lan_id_seq'::regclass)"))
    ade = Column(Float(53))
    gf = Column(Float(53))
    bsg = Column(Float(53))
    rs = Column(String(12))
    ags = Column(String(12))
    sdv_rs = Column(String(12))
    gen = Column(String(50))
    bez = Column(String(50))
    ibz = Column(Float(53))
    bem = Column(String(75))
    nbd = Column(String(4))
    sn_l = Column(String(2))
    sn_r = Column(String(1))
    sn_k = Column(String(2))
    sn_v1 = Column(String(2))
    sn_v2 = Column(String(2))
    sn_g = Column(String(3))
    fk_s3 = Column(String(2))
    nuts = Column(String(5))
    rs_0 = Column(String(12))
    ags_0 = Column(String(12))
    wsk = Column(Date)
    debkg_id = Column(String(16))
geom = Column(Geometry('MULTIPOLYGON', 31467), index=True)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
print(session.query(BkgVg2502Lan).all())
session.close()

In [ ]:
df = pd.DataFrame(session.query(BkgVg2502Lan.id,BkgVg2502Lan.gen, BkgVg2502Lan.reference_date)
                  .filter(BkgVg2502Lan.reference_date == '2016-01-01').all())
df

In [ ]:
print("----> order_by(id):")
query = session.query(BkgVg2502Lan).order_by(BkgVg2502Lan.ags_0)
for _row in query.all():
    print(ags_0)